<a href="https://colab.research.google.com/github/Samaelbeka/georgian_economic_visualizer/blob/main/web_scrapper_gathering_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Starting with playing with python and  gathering all neccessary data


In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import plotly.express as px



url='https://www.geostat.ge/en/modules/categories/23/gross-domestic-product-gdp' # Replace with the URL of the page you want to scrape
response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad status codes (like 404)

soup = BeautifulSoup(response.content, "html.parser")

all_links = soup.find_all('a', href=True)

target_link = None
for link in all_links:
    href = link['href']
    if href.endswith('.xls') or href.endswith('.xlsx'):  # Check for file extension
        if '03_GDP-at-Current-Prices' in href:  # Check for filename
            target_link = href
            break

if target_link:
    print(f"Found the target link: {target_link}")
else:
    print("Target link not found on the page.")



Found the target link: https://geostat.ge/media/67608/03_GDP-at-Current-Prices.xlsx


In [ ]:


# Send a GET request to the file URL
response = requests.get(target_link)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Save the content of the file locally
    with open('01_Output.xlsx', 'wb') as file:
        file.write(response.content)
    print("Excel file downloaded successfully!")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

Excel file downloaded successfully!


In [ ]:
# prompt: show me the shape of downloaded file


# Load the downloaded Excel file into a pandas DataFrame
df = pd.read_excel('01_Output.xlsx',skiprows=1)
df=df.drop(df.columns[0], axis=1)
df = df.reset_index(drop=True)
year_columns = [col for col in df.columns
                if re.search(r'\b(19|20)\d{2}\b', str(col)) is not None]
df=df.drop(columns=year_columns)


In [ ]:


# Function to convert bad date format to datetime objects
def convert_bad_date(bad_date):
    quarter_map = {'I': '03-31', 'II': '06-30', 'III': '09-30', 'IV': '12-31'}

    # Extract quarter and year using regex
    match = re.search(r'([I,II,III,IV]+)\s*(\d+)', bad_date)

    if match:
        quarter = match.group(1).strip()
        year = match.group(2).strip()
        year = '20' + year if len(year) == 2 else year
        date_str = f"{year}-{quarter_map[quarter]}"
        return pd.to_datetime(date_str)
    else:
        return bad_date  # Return original if no match

# ... (Rest of your code, using the updated convert_bad_date function) ...




v=1
for column in df.columns[1:]:
  df.columns.values[v] =convert_bad_date(column)
  v=v+1



In [ ]:
filtered_df = df[df['Economic Activities'].isin(['GDP per capita in GEL', 'GDP per capita, USD','GDP in mil. USD'])]
df_long =filtered_df.melt(id_vars=['Economic Activities'], var_name='Date', value_name='value')
fig = px.line(df_long, x="Date", y="value", color='Economic Activities', title='GDP per capita over time')
fig.show()



Importing Labour Data


In [ ]:

url='https://www.geostat.ge/en/modules/categories/683/Employment-Unemployment' # Replace with the URL of the page you want to scrape
response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad status codes (like 404)

soup = BeautifulSoup(response.content, "html.parser")

all_links = soup.find_all('a', href=True)

target_link = None
for link in all_links:
    href = link['href']
    if href.endswith('.xls') or href.endswith('.xlsx'):  # Check for file extension
        if '01-Labour-Force-Indicators' in href:  # Check for filename
            target_link = href
            break

if target_link:
    print(f"Found the target link: {target_link}")
else:
    print("Target link not found on the page.")





# Send a GET request to the file URL
response = requests.get(target_link)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Save the content of the file locally
    with open('01_Output.xlsx', 'wb') as file:
        file.write(response.content)
    print("Excel file downloaded successfully!")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")



    # prompt: show me the shape of downloaded file


# Load the downloaded Excel file into a pandas DataFrame
df = pd.read_excel('01_Output.xlsx',skiprows=2)

un_emp = df[df['Unnamed: 0'].isin(['Employment rate, percentage','Unemployment rate, percentage','Labour force participation rate, percentage'])]
print(un_emp)



v=1
for column in un_emp.columns[1:]:
  un_emp.columns.values[v] =   pd.to_datetime(un_emp.columns[v], format='%Y')
  v=v+1


un_emp =un_emp.melt(id_vars=['Unnamed: 0'], var_name='Date', value_name='value')

un_emp.columns.values[0]='Indicator'



fig = px.line(un_emp, x="Date", y="value", color='Indicator', title='Labor Data')
fig.show()


Found the target link: https://geostat.ge/media/68496/01-Labour-Force-Indicators.xlsx
Excel file downloaded successfully!
                                     Unnamed: 0       1998       1999  \
8                 Unemployment rate, percentage  12.380997  12.627719   
9   Labour force participation rate, percentage  65.393801  64.083561   
10                  Employment rate, percentage  57.297397  55.991269   

         2000       2001       2002       2003       2004       2005  \
8   10.345920  11.147750  13.509983  12.680092  13.865755  15.115243   
9   65.229684  66.226908  63.906144  64.790124  63.601834  62.737230   
10  58.481073  58.844098  55.272435  56.574676  54.782960  53.254345   

         2006  ...       2015       2016       2017       2018       2019  \
8   15.371588  ...  21.911556  21.722089  21.597623  19.249060  17.604935   
9   60.545615  ...  55.500420  54.952375  54.489056  52.903787  51.785442   
10  51.238793  ...  43.339415  43.015571  42.720715  42.720305  4

Timestamp('1999-01-01 00:00:00')